In [1]:
 #load Raw data from TDT ECoG maschine, plz run ""
import numpy as np
import scipy.io as scio
from scipy import signal
import tdt
import os
import  wave
import matplotlib.pyplot as plt
import json
from scipy.fftpack import fft
from random import shuffle
import h5py
import scipy.io as scio
import scipy.io.wavfile
import math
import mne
import os
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor
import torch
from torch.utils import data as Data
import torch.nn as nn
import seaborn as sns
from ecog_band import EcogBandRes
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset,Dataset
from ecog_band.solver import Nfold_solver
from ecog_band.models import ECOGRes50_feature,ECOGRes50
from ecog_band.datasetSigband import CustomDatasetSigband
from ecog_band.utils import *


/root/pp/covert-reading/Ecog_pretrain/ecog_band/solver.py:8: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm  # may raise warning about Jupyter


In [2]:
bands_list=['else1','delta','theta','alpha','beta', 'gamma','high gamma','else2']
# bands_list=['gamma','high gamma']
HS=86
freq=500
elec=7
band = 'All_bands'

best_fold_model_path = '/root/pp/covert-reading/Ecog_pretrain/best_models'
y_save_path = f'/root/pp/covert-reading/Ecog_pretrain/Bclassification_results_resnet_shuffle/HS{HS}/{freq}/{elec}'
contribution_save_path = f'/root/pp/covert-reading/Ecog_pretrain/Bclassification_results_resnet_shuffle/HS{HS}/{freq}/{elec}'

os.makedirs(y_save_path, exist_ok=True)

# all band do binary classification

In [3]:
# all band do binary classification
band = 'All_bands'
path_elec=f'/public/DATA/overt_reading/dataset/HS86/500/{elec}'
num_samples=len(os.listdir(path_elec)) # 12
# print(num_samples)
data_loader = CustomDatasetSigband(HS, path_elec, freq, elec, num_samples, exclude=True)
net=ECOGRes50_feature()
for batch in data_loader:
    print(batch[0].shape,batch[1]) #[1, 1, 375] 0
    break

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.95)

solver = Nfold_solver(
    model = net,
    optimizer = optimizer,
    criterion = nn.BCEWithLogitsLoss(),
    lr_scheduler = lr_scheduler,
    )
# print(net)

fold_results, fold_all_labels, fold_all_preds = solver.train(
            epochs=10,
            data_loader=data_loader,
            band=band,
            best_fold_model_path=os.path.join(best_fold_model_path,str(elec))
            )
# band_acc_list[band_each] = fold_results # fold_results is a list, include 5 fold accuracy, every fold accuracy is the best accuracy in validate epochs
# plt confusion matrix

all_y_true = [label for sublist in fold_all_labels for label in sublist] # merge 5 list to one
all_y_pred = [label for sublist in fold_all_preds for label in sublist] # merge 5 list to one
np.save(os.path.join(y_save_path, f'{band}_y_pred.npy'), all_y_pred)
np.save(os.path.join(y_save_path, f'{band}_y_true.npy'), all_y_pred)


torch.Size([1, 501, 375]) tensor(0)
Fold 1


[Train] Epoch 1/10:   0%|          | 0/576 [00:00<?, ?block/s]

# do binary classification on each band

In [ ]:
# five fold for each band
band_acc_list = {} # 存储每一个band的五次交叉验证最好的结果

for band_each in bands_list:
    path_elec=f'/public/DATA/overt_reading/dataset/HS86/500/{elec}'
    num_samples=len(os.listdir(path_elec)) # 12
    # print(num_samples)
    data_loader = CustomDatasetSigband(HS, path_elec, freq, elec, num_samples, band_each, exclude=True)
    for batch in data_loader:
        print(batch[0].shape,batch[1]) #[1, 1, 375] 0
        break

    net=ECOGRes50_feature()

    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.95)

    solver = Nfold_solver(
        model = net,
        optimizer = optimizer,
        criterion = nn.BCEWithLogitsLoss(),
        lr_scheduler = lr_scheduler,
        )
    # print(net)

    fold_results, fold_all_labels, fold_all_preds = solver.train(
                epochs=10,
                data_loader=data_loader,
                band=band_each,
                best_fold_model_path=os.path.join(best_fold_model_path,str(elec))
                )
    band_acc_list[band_each] = fold_results # fold_results is a list, include 5 fold accuracy, every fold accuracy is the best accuracy in validate epochs
    # plt confusion matrix
    all_y_true = [label for sublist in fold_all_labels for label in sublist] # merge 5 list to one
    all_y_pred = [label for sublist in fold_all_preds for label in sublist] # merge 5 list to one
    np.save(os.path.join(y_save_path, f'{band_each}_y_pred.npy'), all_y_pred)
    np.save(os.path.join(y_save_path, f'{band_each}_y_true.npy'), all_y_true)
    plt_confusion_matrix_sum(all_y_true, all_y_pred, band_each)
    
pltbox_band_five_cross_validation(band_acc_list)
# plt_band_acc(band_acc_list)

# write fold resultes into file
# save_path = f'/root/pp/covert-reading/Ecog_pretrain/fold_results/HS{HS}/{freq}/{elec}'
# save_band_classify_acc(band_acc_list, save_path)


In [ ]:
# show accuracy for each band
from ecog_band.utils import pltbar_accuracy
accuracy = []
for band in bands_list:
    accuracy.append(cal_acc_band1_from_y(y_save_path, band))
pltbar_accuracy(accuracy, bands_list, removed=True)

# calculate contribution for each band

In [ ]:
# calculate contribution for each band
baseline_accuracy = cal_acc_band1_from_y(y_save_path, 'All_bands')
contributions_list = []

for band in bands_list:
    # print(band)
    badn_acc = cal_acc_band1_from_y(y_save_path, band)
    contribution = baseline_accuracy - badn_acc
    print(f'band: {band}, contribution_acc:{contribution}')
    contributions_list.append(contribution)

np.save(os.path.join(contribution_save_path, f'contributions.npy'), contributions_list)

In [ ]:
# plt contribution bar
contribution_list = np.load(os.path.join(contribution_save_path, f'contributions.npy'))
pltbar_contribution(contribution_list, bands_list)

In [ ]:
# plot all confusion matrix on one graph
cm = {}
for band in bands_list:
    cm[band] = cal_cm_band1_from_y(y_save_path, band)

plt_allband_confusion_matrices(cm, bands_list)